In [1]:
import os, gc
from tqdm import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
#from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

import torch

import warnings
warnings.filterwarnings('ignore')

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
NVIDIA GeForce RTX 3080 Ti


In [2]:
loader = PyPDFLoader("pdf_samples/1716891429607.pdf")
pages = loader.load_and_split()

print(pages[11].page_content)

II БҮЛЭГ.ӨГӨГДЛИЙН ХҮСНЭГТ
ТШЗ-ны 
холбогдох 
заалтА.ЕРӨНХИЙ ЗҮЙЛ
ТШЗ 1.1Захиалагчийн нэр: Монгол шуудан ХК
ТШЗ 1.1Зарлалын дугаар: МШ/20240202070
Тендер шалгаруулалтын нэр: Принтерийн хор
Тендер шалгаруулалтын багцын тоо болон нэр: Байхгүй
ТШЗ 2.1Санхүүжилтийн эх үүсвэр: Өөрийн хөрөнгө
Нийт төсөвт өртөг: 43,000,000 /дөчин гурван сая / төгрөг
Б.АГУУЛГА
ТШЗ 9.2Тендерийн баримт бичгийн талаар тодруулга авах хаяг:
Тендерийн баримт бичгийн талаар тодруулга авах хүсэлтийг хүлээн авах  
хугацааны хязгаар тендер хүлээн авах эцсийн хугацаанаас ажлын 2-оос  
доошгүй  хоногийн  өмнө  байна.  Тодруулга,  нэмэлт  мэдээлэл  авах  
хүсэлтийн хариуг хүсэлт гаргагчийн нэрийг дурдалгүйгээр албан бичгээр  
гаргаж,  түүний  хуулбарыг  Төрийн  худалдан  авах  ажиллагааны  цахим  
систем  (www.tender.gov.mn)-д  нийтлэх  бөгөөд  тус  тодруулга,  нэмэлт  
мэдээллийг бүх тендерт оролцогчдод нэгэн зэрэг хүргүүлсэнд тооцно.  
Иймд тендерт оролцохыг сонирхогч этгээдүүд дээрх цахим систем дэх  
тендерийн  урилгын

In [3]:
docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs_split = r_splitter.split_documents(docs)

In [4]:
embedding = HuggingFaceEmbeddings(
    #model_name = "tugstugi/bert-base-mongolian-cased",
    #model_name = "sentence-transformers/LaBSE",
    model_name = "bayartsogt/albert-mongolian",
    model_kwargs = {"device": "cuda"}, 
    encode_kwargs = {"normalize_embeddings": True}
)

No sentence-transformers model found with name bayartsogt/albert-mongolian. Creating a new one with mean pooling.


# Chroma test

In [5]:
vector_store = Chroma.from_documents(documents = docs_split, 
                                     embedding = embedding,
                                     persist_directory = 'db/')

In [6]:
prompt = 'санхүүгийн чадавхын шаардлага хэд вэ'

docs = vector_store.similarity_search(prompt, k = 10)

for idx, doc in enumerate(docs):
    print(f"Returned: {idx + 1}")
    print(doc.page_content, '\n')

Returned: 1
баталгаажуулсан байна.
6.Бараанд  тавигдах  6.1.Дотоодын үйлдвэрлэлээр хангах боломжтой чанар, 

Returned: 2
баталгаажуулсан байна.
6.Бараанд  тавигдах  6.1.Дотоодын үйлдвэрлэлээр хангах боломжтой чанар, 

Returned: 3
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 4
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 5
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 

Returned: 6
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 

Returned: 7
үзүүлэлтэд  үндэслэн  жишиг  маягтуудын  хаалт  дотор  налууга

In [7]:
print(docs[8].page_content)

дээш хувийг хангасан байна.
18.Санхүүгийн 
чадавхын шаардлага18.1.Хуулийн  16.5-д  зааснаар  оролцогч  
өгөгдлийн  хүснэгтэд  заасан  санхүүгийн  чадавхын  
шаардлага хангасан байна.
18.2.Санхүүгийн  чадавхын  шаардлагыг  хангах  нотлох  
баримт  ирүүлэхийг  шаардах  бол  хуулийн  16.2,  16.4-т  
заасан баримт бичгээс сонгож өгөгдлийн хүснэгтэд заана.
18.3.Оролцогчийн дансны мэдээлэл, түүний үлдэгдлийн  
талаар  харилцагч  банкнаас  мэдэгдсэнээс  бусад  банк,


In [8]:
print(docs[9].page_content)

дээш хувийг хангасан байна.
18.Санхүүгийн 
чадавхын шаардлага18.1.Хуулийн  16.5-д  зааснаар  оролцогч  
өгөгдлийн  хүснэгтэд  заасан  санхүүгийн  чадавхын  
шаардлага хангасан байна.
18.2.Санхүүгийн  чадавхын  шаардлагыг  хангах  нотлох  
баримт  ирүүлэхийг  шаардах  бол  хуулийн  16.2,  16.4-т  
заасан баримт бичгээс сонгож өгөгдлийн хүснэгтэд заана.
18.3.Оролцогчийн дансны мэдээлэл, түүний үлдэгдлийн  
талаар  харилцагч  банкнаас  мэдэгдсэнээс  бусад  банк,


# FAISS test

In [9]:
faiss_index = FAISS.from_documents(documents = docs_split, 
                                   embedding = embedding)

In [10]:
# Энд 6 болон 8 дахь үр дүнгүүд ойр харагдаж байна
docs = faiss_index.similarity_search(prompt, k = 10)
for idx, doc in enumerate(docs):
    print(f"Returned: {idx + 1}")
    print(doc.page_content, '\n')

Returned: 1
баталгаажуулсан байна.
6.Бараанд  тавигдах  6.1.Дотоодын үйлдвэрлэлээр хангах боломжтой чанар, 

Returned: 2
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 3
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 4
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 

Returned: 5
үзүүлэлтэд  үндэслэн  жишиг  маягтуудын  хаалт  дотор  налуугаар  бичсэн  тохирох  


In [11]:
print(docs[5].page_content)

дээш хувийг хангасан байна.
18.Санхүүгийн 
чадавхын шаардлага18.1.Хуулийн  16.5-д  зааснаар  оролцогч  
өгөгдлийн  хүснэгтэд  заасан  санхүүгийн  чадавхын  
шаардлага хангасан байна.
18.2.Санхүүгийн  чадавхын  шаардлагыг  хангах  нотлох  
баримт  ирүүлэхийг  шаардах  бол  хуулийн  16.2,  16.4-т  
заасан баримт бичгээс сонгож өгөгдлийн хүснэгтэд заана.
18.3.Оролцогчийн дансны мэдээлэл, түүний үлдэгдлийн  
талаар  харилцагч  банкнаас  мэдэгдсэнээс  бусад  банк,


In [13]:
print(docs[7].page_content)

хугацаанд  цахим  системээс  холбогдох  зааварчилгааны  дагуу  
авсан байх,/
Дээр  заасан  баримт  бичгүүдийн  аль  нэгийг  ирүүлээгүй  эсвэл  
шаардлага хангаагүй тохиолдолд тендерээс татгалзах үндэслэл  
болно.
ТШЗ 18.1Санхүүгийн чадавхын шаардлага: Шаардана
1.Борлуулалтын орлогын хэмжээ : 2022, 2023 онуудын жил тус  
бүрийн  борлуулалтын  орлогын  хэмжээ  нь  батлагдсан  төсөвт  
өртгийн 100 хувь түүнээс багагүй байна,
2.Түргэн  хөрвөх  хөрөнгөтэй  холбогдох  шаардлага :  Нийт


## Эндээс харахад 2 санаа төрж байна

1. Мэдээж хэрэг хуваалтын хэмжээг багассах (магадгүй chunk_size = 5000, 10000 болгох) мөн хуваалт хоорондын давхцлыг ихэсгэх
2. Prompt-г илүү нарийн болгох

# 2. Prompt-г илүү нарийн болгох

Эхлээд уг замаар явж үзье

In [30]:
def get_docs(prompt, vector_db = 'chroma', k = 10):
    if vector_db == 'chroma':
        return vector_store.similarity_search(prompt, k)
    return faiss_index.similarity_search(prompt, k)

def get_and_print_docs(prompt, vector_db, k):
    docs = get_docs(prompt, vector_db, k)
    print(f"Prompt: {prompt}")
    print(f"Vector DB: {vector_db}\n")

    for idx, doc in enumerate(docs):
        print(f"Returned: {idx + 1}")
        print(doc.page_content, '\n')

    return docs

### FAISS

In [22]:
prompt = 'санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ'

docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ
Vector DB: faiss

Returned: 1
Нууцалсан баримт бичиг,  
мэдээлэлНууцалсан хууль зүйн үндэслэл
1. 

Returned: 2
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 3
бичгийн бүрдэл болохгүй.
8.3.Сонирхогч этгээд тендер шалгаруулалтын баримт  
бичгийн шаардлага, шалгуур үзүүлэлт, гэрээний нөхцөл,  
тендерийн жишиг маягт, техникийн тодорхойлолт болон  
тэдгээрт иш татсан хууль, журам, зааврыг судалж, түүнд  
нийцүүлэн тендер бэлтгэнэ.
9.Тендер 
шалгаруулалтын  баримт  
бичгийн талаар тодруулга  
авах9.1.Сонирхогч этгээд тендер шалгаруулалты

In [23]:
print(docs[5].page_content)

хугацаанд  цахим  системээс  холбогдох  зааварчилгааны  дагуу  
авсан байх,/
Дээр  заасан  баримт  бичгүүдийн  аль  нэгийг  ирүүлээгүй  эсвэл  
шаардлага хангаагүй тохиолдолд тендерээс татгалзах үндэслэл  
болно.
ТШЗ 18.1Санхүүгийн чадавхын шаардлага: Шаардана
1.Борлуулалтын орлогын хэмжээ : 2022, 2023 онуудын жил тус  
бүрийн  борлуулалтын  орлогын  хэмжээ  нь  батлагдсан  төсөвт  
өртгийн 100 хувь түүнээс багагүй байна,
2.Түргэн  хөрвөх  хөрөнгөтэй  холбогдох  шаардлага :  Нийт


In [24]:
prompt = 'санхүүгийн чадавхийн шаардлагад орлогын хэмжээ ямар байх вэ'

docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад орлогын хэмжээ ямар байх вэ
Vector DB: faiss

Returned: 1
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 

Returned: 2
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 3
дасан зохицох, ногоон орчныг бүрдүүлэхэд чиглэсэн байх үзүүлэлтийг тусгаж  
өгнө.
Тендерийн баримт бичигт зааснаас бусад тохиолдолд бүх бараа, материал нь  
шинэ, ашиглагдаагүй, сүүлийн үеийн технологи зэргийг хангасан байх нөхцөлийг  
шаардана. Өмнө хэрэгжсэн, ижил төстэй төслийн техникийн тодорхойлолтыг  
жишээ болгон ашиглах нь давуу талтай.
Техникийн  тодорхойлолтын  шалгуур  үзүүлэлтүүд  нь  хязгаарласан  шинжтэй 

Returned: 4
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй ч

In [25]:
print(docs[6].page_content)

хугацаанд  цахим  системээс  холбогдох  зааварчилгааны  дагуу  
авсан байх,/
Дээр  заасан  баримт  бичгүүдийн  аль  нэгийг  ирүүлээгүй  эсвэл  
шаардлага хангаагүй тохиолдолд тендерээс татгалзах үндэслэл  
болно.
ТШЗ 18.1Санхүүгийн чадавхын шаардлага: Шаардана
1.Борлуулалтын орлогын хэмжээ : 2022, 2023 онуудын жил тус  
бүрийн  борлуулалтын  орлогын  хэмжээ  нь  батлагдсан  төсөвт  
өртгийн 100 хувь түүнээс багагүй байна,
2.Түргэн  хөрвөх  хөрөнгөтэй  холбогдох  шаардлага :  Нийт


In [27]:
# Nope, no dice
prompt = 'санхүүгийн чадавхийн шаардлагад түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'

docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: faiss

Returned: 1
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 2
асуудлаар  Захиалагчтай  харилцах  эрхтэй  
Нийлүүлэгчээс томилогдсон этгээдийг;
1.1.14.“Гэнэтийн буюу давагдашгүй хүчний шинжтэй  
онцгой нөхцөл байдал“ гэж талуудын хараа хяналтаас  
гадуур  болсон,  урьдчилан  таамаглах  боломжгүй, 

Returned: 3
үзүүлэлтэд  үндэслэн  жишиг  маягтуудын  хаалт  дотор  налуугаар  бичсэн  тохирох  
мэдээллийг зааварт нийцүүлэн бүрэн, үнэн зөв бөглөж ирүүлнэ.Гэрээний нөхцөл бүлэг нь  
захиалагч

In [31]:
prompt = 'түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'

docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 20)

Prompt: түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: faiss

Returned: 1
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 2
санаа өгөх зорилгоор оруулсан ба тухайн гэрээний онцлогийг харгалзан захиалагч  
эцэслэнэ. 

Returned: 3
ирүүлнэ. Энэхүү тодорхойлолтыг ирүүлээгүй тохиолдолд тендер  
нээсэн өдрийн байдлаар татвар, нийгмийн даатгалын шимтгэлийн  
хугацаа хэтэрсэн өр төлбөрийн талаарх мэдээллийг шалгана
3.Тендер  шалгаруулалт  зарласнаас  хойш  хугацаанд  цахим  
системээс  авсан  нийгмийн  даатгалын  шимтгэлийн  төлбөргүй  
тодорхойлолтыг  цахим  системийн  холбогдох  талба

### Chroma

In [32]:
prompt = 'санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ'

docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ
Vector DB: chroma

Returned: 1
Нууцалсан баримт бичиг,  
мэдээлэлНууцалсан хууль зүйн үндэслэл
1. 

Returned: 2
Нууцалсан баримт бичиг,  
мэдээлэлНууцалсан хууль зүйн үндэслэл
1. 

Returned: 3
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 4
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардаг

In [34]:
prompt = 'санхүүгийн чадавхийн шаардлагад түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'

docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: chroma

Returned: 1
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 2
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 


In [36]:
prompt = 'түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'

docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: chroma

Returned: 1
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 2
тавьсан  шаардлагад  нийцсэн,  бодитой  бөгөөд  өрсөлдөх  чадвартай  тендер  
бэлтгэхэд оролцогчдод зайлшгүй чухал нөхцөл болно. 
Техникийн  тодорхойлолтыг  бэлтгэхэд  өрсөлдөөнийг  өрнүүлэх  зарчмыг  
баримтлах ёстой бөгөөд худалдан авах барааны шинж чанар, материал, үйл  
ажиллагааны  шаардагдах  стандартын  талаар  тодорхой  тусгана.  Эдгээрийг  
хангасан  нөхцөлд  худалдан  авах  ажиллагааны  хэмнэлттэй,  үр  ашигтай, 

Returned: 3
санаа өгөх зорилгоо

# 1. Хуваалтын тоог багассах

In [50]:
docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 500)
docs_split = r_splitter.split_documents(docs)

In [51]:
vector_store = Chroma.from_documents(documents = docs_split, 
                                     embedding = embedding)

faiss_index = FAISS.from_documents(documents = docs_split, 
                                   embedding = embedding)

In [56]:
def get_docs(prompt, vector_db = 'chroma', k = 10):
    if vector_db == 'chroma':
        return vector_store.similarity_search(prompt, k)
    return faiss_index.similarity_search(prompt, k)

def get_and_print_docs(prompt, vector_db, k):
    docs = get_docs(prompt, vector_db, k)
    print(f"Prompt: {prompt}")
    print(f"Vector DB: {vector_db}\n")

    for idx, doc in enumerate(docs):
        print(f"Returned: {idx + 1}")
        print(doc.page_content, '\n')

    return docs

prompt = 'санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ'

# nahhh
docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ
Vector DB: chroma

Returned: 1
БАРАА НИЙЛҮҮЛЭЛТИЙН ХУВААРЬ
[Захиалагч доорх хүснэгтийн 1-6 хүртэлх баганыг, тендерт оролцогч 7 дугаар баганыг бөглөнө ]
№Барааны нэр Тоо 
хэмжээХэмжих 
нэгж, тунБарааг 
хүргэх 
эцсийн цэгБараа нийлүүлэх хугацаа
Захиалагч бараа хүлээн авах  
хугацаа 
(эхлэх болон дуусах хугацаа)Нийлүүлэгчийн санал болгох  
хугацаа
(Оролцогч энэ баганыг бөглөнө )
12345 6 7
1Принтерийн 
хор1415ширхэгУБ, ЧД, 1-р 
хороо, Төв 
шууданГэрээ байгуулснаас хойш 1 жилийн  
хугацаанд захиалагчийн хэрэгцээ  
шаардлагын дагуу нийлүүлэх[6 дугаар баганад заасан хугацаанд  
багтаан нийлүүлэхээр санал болгож  
буй хугацааг бичих]
2
3
… 

Returned: 2
ХАРЬЦУУЛАЛТЫН АРГААР 
БАРАА ХУДАЛДАН АВАХ 
ТЕНДЕР ШАЛГАРУУЛАЛТЫН 
БАРИМТ БИЧИГ
Тендер 
шалгаруулалтын 
нэр:[Принтерийн хор ]
Тендер 
шалгаруулалтын 
дугаар:[МШ/20240202070] 

Returned: 3
Техникийн тодорхойлолт
      1. Нийлүүлэгдэх бараа нь Монгол улсын чанарын шаардлага ханг

In [57]:
# nahhh
docs = get_and_print_docs(prompt, vector_db = 'faiss', k = 10)

Prompt: санхүүгийн чадавхийн шаардлагад ямар баримт бичиг хамаарах вэ
Vector DB: faiss

Returned: 1
БАРАА НИЙЛҮҮЛЭЛТИЙН ХУВААРЬ
[Захиалагч доорх хүснэгтийн 1-6 хүртэлх баганыг, тендерт оролцогч 7 дугаар баганыг бөглөнө ]
№Барааны нэр Тоо 
хэмжээХэмжих 
нэгж, тунБарааг 
хүргэх 
эцсийн цэгБараа нийлүүлэх хугацаа
Захиалагч бараа хүлээн авах  
хугацаа 
(эхлэх болон дуусах хугацаа)Нийлүүлэгчийн санал болгох  
хугацаа
(Оролцогч энэ баганыг бөглөнө )
12345 6 7
1Принтерийн 
хор1415ширхэгУБ, ЧД, 1-р 
хороо, Төв 
шууданГэрээ байгуулснаас хойш 1 жилийн  
хугацаанд захиалагчийн хэрэгцээ  
шаардлагын дагуу нийлүүлэх[6 дугаар баганад заасан хугацаанд  
багтаан нийлүүлэхээр санал болгож  
буй хугацааг бичих]
2
3
… 

Returned: 2
ХАРЬЦУУЛАЛТЫН АРГААР 
БАРАА ХУДАЛДАН АВАХ 
ТЕНДЕР ШАЛГАРУУЛАЛТЫН 
БАРИМТ БИЧИГ
Тендер 
шалгаруулалтын 
нэр:[Принтерийн хор ]
Тендер 
шалгаруулалтын 
дугаар:[МШ/20240202070] 

Returned: 3
Техникийн тодорхойлолт
      1. Нийлүүлэгдэх бараа нь Монгол улсын чанарын шаардлага ханга

In [58]:
prompt = 'түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ'

docs = get_and_print_docs(prompt, vector_db = 'chroma', k = 10)

Prompt: түргэн хөрвөх хөрөнгөтэй холбогдох ямар шаардлага байгаа вэ
Vector DB: chroma

Returned: 1
БАРАА НИЙЛҮҮЛЭЛТИЙН ХУВААРЬ
[Захиалагч доорх хүснэгтийн 1-6 хүртэлх баганыг, тендерт оролцогч 7 дугаар баганыг бөглөнө ]
№Барааны нэр Тоо 
хэмжээХэмжих 
нэгж, тунБарааг 
хүргэх 
эцсийн цэгБараа нийлүүлэх хугацаа
Захиалагч бараа хүлээн авах  
хугацаа 
(эхлэх болон дуусах хугацаа)Нийлүүлэгчийн санал болгох  
хугацаа
(Оролцогч энэ баганыг бөглөнө )
12345 6 7
1Принтерийн 
хор1415ширхэгУБ, ЧД, 1-р 
хороо, Төв 
шууданГэрээ байгуулснаас хойш 1 жилийн  
хугацаанд захиалагчийн хэрэгцээ  
шаардлагын дагуу нийлүүлэх[6 дугаар баганад заасан хугацаанд  
багтаан нийлүүлэхээр санал болгож  
буй хугацааг бичих]
2
3
… 

Returned: 2
Техникийн тодорхойлолт
      1. Нийлүүлэгдэх бараа нь Монгол улсын чанарын шаардлага хангасан,  
лабраторын  шинжилгээний  дүгнэлттэй,  албан  ёсны   дистрибьютерийн  
/гэрээт борлуулагч/ эрхтэйгээр борлуулдаг байх. 
     2. Хүргэлтийг нийлүүлэгч компани өөрөө хариуцах ба захиа